# Linear Classification and Nearest Neighbor Classification

In [15]:
import sys
import csv
import math
import numpy as np
from numpy import matrix, polyfit
from numpy.linalg import inv
from numpy.random.mtrand import shuffle

def loadCovMatrix(fileName):
    fp  = open(fileName)
    cov= [m.strip() for line in fp.readlines() for m in line.split(',') if m.strip()]
    cov = list(map(lambda v: float(v), cov))
    covM = np.array([cov])
    covM.shape = (20,20)
    #print covM
    return covM

def loadMeanVector(fileName):
    fp  = open(fileName)
    m_i= [m.strip() for line in fp.readlines() for m in line.split(',') if m.strip()]
    m_i = list(map(lambda v: float(v), m_i))
    #print np.array([m_i]).transpose()
    return np.array([m_i]).transpose()
    
m_0 = loadMeanVector('DS1_m_0.txt')
m_1 = loadMeanVector('DS1_m_1.txt')
cov = loadCovMatrix('DS1_Cov.txt')
#P(C0) = P(C1) = 0.5
pCi = 0.5
n = 2400

# P(X|Ci)
def pXGivenCi(x,mean,cov):
    det_cov = np.linalg.det(cov)
    #print det_cov
    k = x.shape[0]
    coef = 1.0/( math.sqrt(2.0*pow(math.pi,k)*det_cov) )
    #print 2.0*pow(math.pi,k) *det_cov
    x_minus_mean = np.subtract(x,mean)
    x_minus_mean_T = x_minus_mean.transpose()
    
    exp = math.exp((-0.5)*(x_minus_mean_T.dot(inv(cov))).dot(x_minus_mean))
    #print exp
    
    return coef*exp
# P(Ci and X)    
def pCiAndX(x,mean,cov):
    return pCi * pXGivenCi(x,mean,cov)

# P(X)
def pX(x,meanA,meanB,cov):
    return pCiAndX(x,meanA,cov) + pCiAndX(x,meanB,cov)

# P(Ci | X ) = P(Ci and X)/P(X)
def pCiGivenX(x,meanA,meanB,cov):
    return pCiAndX(x, meanA, cov)/pX(x, meanA, meanB, cov)

'''Test pdf
x = np.array([[5],[2],[3]])
u = np.array([[1],[4],[3]])
X_minus_u = np.subtract(x,u)
X_minus_u_T = X_minus_u.transpose()
cov = np.array([ [1,2,3],[1,6,7],[3,4,9] ])
det_cov = np.linalg.det(cov)
k = x.shape[0]
a = 1.0/(math.sqrt(2.0*pow(math.pi,k)*det_cov))
'''


#Shuffle matrix X and matrix Y in the same fashion
def shuffleData(matrix_X,matrix_Y):
    rand_train = np.arange(len(matrix_X))
    np.random.shuffle(rand_train)
    matrix_X = matrix_X[rand_train]
    matrix_Y= matrix_Y[rand_train]
    return {'x':matrix_X,'y':matrix_Y}
    

#Return 3 sets of data: TRAIN, TEST and VALID
def generateData(u_0,u_1,cov,numData):

    #Initialize matrices
    matrix_X0 = np.array([])
    matrix_Y0 = np.array([])
    matrix_X1 = np.array([])
    matrix_Y1 = np.array([])
    
    # Now generate 2000 examples for each class
    # X = [x1=rand(1.0,2.3), x2=rand(1.0,2,3).....,x20 = rand(1.0,2.3)]
    halfData = numData/2
    while len(matrix_X0) < halfData or len(matrix_X1) < halfData:
        x = np.array([np.random.uniform(0,4,20)]).transpose()
        a = pXGivenCi(x,u_0,cov)
        b = pXGivenCi(x,u_1,cov)
        
        #LABEL DATA, -1 IF NEGATIVE ELSE 1
        if(a >= b and len(matrix_X0) < halfData):
            if(len(matrix_X0) == 0):
                matrix_X0 = x.transpose()
                matrix_Y0 = np.array([[-1]])
            else: 
                matrix_X0 = np.concatenate((matrix_X0,x.transpose())) 
                matrix_Y0 = np.concatenate((matrix_Y0,np.array([[-1]])))
        elif(b > a and len(matrix_X1) < halfData):
            if(len(matrix_X1) == 0): 
                matrix_X1 = x.transpose()
                matrix_Y1 = np.array([[1]])
            else: 
                matrix_X1 = np.concatenate((matrix_X1,x.transpose())) 
                matrix_Y1 = np.concatenate((matrix_Y1,np.array([[1]])))
    
    #Train 60% = 30%* + 30%
    #Test 20% = 10% + 10%
    #Valid 20% = 10% + 10%
    
    trainUpTo = (int)(0.6*halfData)
    testUpTo = (int)(0.2*halfData + trainUpTo)
    validUpTo = (int)(0.2*halfData + testUpTo)
    
    #Data separation
    #first 1200 x's of matrix_X0 + first 1200 x's of matrix_X1 = 2400 TRAIN
    matrix_X_train = np.concatenate((np.array(matrix_X0[0:trainUpTo]),np.array(matrix_X1[0:trainUpTo])))
    matrix_Y_train = np.concatenate((np.array(matrix_Y0[0:trainUpTo]),np.array(matrix_Y1[0:trainUpTo])))
    
    #1200th-1600 of matrix_X0 + 1200th-1600th of matrix_X = 800 TEST
    matrix_X_test = np.concatenate((np.array(matrix_X0[trainUpTo:testUpTo]),np.array(matrix_X1[trainUpTo:testUpTo])))
    matrix_Y_test = np.concatenate((np.array(matrix_Y0[trainUpTo:testUpTo]),np.array(matrix_Y1[trainUpTo:testUpTo])))
    
    #1600th-2000th of matrix_Y0 + 1600th-2000th of matrix_X1 = 800 VALID
    matrix_X_valid = np.concatenate((np.array(matrix_X0[testUpTo:validUpTo]),np.array(matrix_X1[testUpTo:validUpTo])))
    matrix_Y_valid = np.concatenate((np.array(matrix_Y0[testUpTo:validUpTo]),np.array(matrix_Y1[testUpTo:validUpTo])))
    
    trainShuffled = shuffleData(matrix_X_train,matrix_Y_train)
    testShuffled = shuffleData(matrix_X_test, matrix_Y_test)
    validShuffled = shuffleData(matrix_X_valid,matrix_Y_valid)
    
    return {'train':(trainShuffled['x'],trainShuffled['y']), 
            'test': (testShuffled['x'],testShuffled['y']),
            'valid':(validShuffled['x'], validShuffled['y'])}

data = generateData(m_0, m_1, cov, 4000)

matrix_X_train = data['train'][0]
matrix_Y_train = data['train'][1]
matrix_X_test = data['test'][0]
matrix_Y_test = data['test'][1]
matrix_X_valid = data['valid'][0]
matrix_Y_valid = data['valid'][1]

#Extract matrix to csv files 
def writeMatrixToCSV(data,fileName):
    with open(fileName, 'w') as new_file:
        csv_writer = csv.writer(new_file)
        for line in data:
            csv_writer.writerow(line)


#first, combine matrix_X with matrix_Y so that the label is in the last column
data_train = np.column_stack((matrix_X_train, matrix_Y_train.transpose()[0]))
data_test = np.column_stack((matrix_X_test, matrix_Y_test.transpose()[0]))
data_valid = np.column_stack((matrix_X_valid, matrix_Y_valid.transpose()[0]))

#### UNCOMMENT THIS CODE TO GET A NEW RANDOM DATA SET #####
writeMatrixToCSV(data_train,'DS1_train.csv')
writeMatrixToCSV(data_test,'DS1_test.csv')
writeMatrixToCSV(data_valid,'DS1_valid.csv')

In [24]:
### LOAD THE MATRICES FROM CSV FILES ###

def loadMatrix(fileName):
    with open(fileName,'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        data = np.array(list(csv_reader))
        matrix_X = data[:,0:20].astype(np.float)
        matrix_Y = np.array([data[:,20].astype(np.float)]).transpose()
    return {'x': matrix_X, 'y': matrix_Y}

# Load matrix_X_train from csv file (extracted from Q1)
matrix_X_train = loadMatrix('DS1_train.csv').get('x');

# Load matrix_Y_train from csv file
matrix_Y_train = loadMatrix('DS1_train.csv').get('y');

# Load matrix_X_valid from csv file
matrix_X_valid = loadMatrix('DS1_valid.csv').get('x');

# Load matrix_Y_valid from csv file
matrix_Y_valid = loadMatrix('DS1_valid.csv').get('y');

# Load matrix_X_test from csv file
matrix_X_test = loadMatrix('DS1_test.csv').get('x');

# Load matrix_Y_test from csv file
matrix_Y_test = loadMatrix('DS1_test.csv').get('y');

##### Q2 MLE


def linearClassifier(matrix_X_train, matrix_Y_train, matrix_X_test,matrix_Y_test):
    #FIND U1,U2
    sumV0 = np.array([[0]*20])
    sumV1 = np.array([[0]*20])
    
    for i in range(len(matrix_X_train)):
        x = matrix_X_train[i]
        if matrix_Y_train[i] == -1:
            sumV0 = sumV0 + x
        else:
            sumV1 = sumV1 + x
    
    u0 = ((1.0/1200) * sumV0).transpose()
    u1 = ((1.0/1200) * sumV1).transpose()
    #print u0
    #print u1
    #print np.subtract(matrix_X_train[0].transpose(),u1).shape
    truePos = 0
    trueNeg = 0
    falsePos = 0
    falseNeg = 0
    
    #FIND COV
    S0 = np.array([])
    S1 = np.array([])
    for i in range(len(matrix_X_train)):
        x = matrix_X_train[i].transpose()
        #x = np.array([matrix_X_train[i]]).transpose()
        #print x
        if matrix_Y_train[i] == -1:
            if(len(S0) == 0): S0 = (x-u0).dot((x-u0).transpose())
            else: 
                S0 = S0 + (x-u0).dot((x-u0).transpose())
                #print S0
        else:
            if(len(S1) == 0): S1 = (x-u1).dot((x-u1).transpose())
            else: S1 = S1 + (x-u1).dot((x-u1).transpose())
    #print S0
    covMatrix = (1.0/5000)* (S0+S1)
    truePos = 1.8*n
    #print covMatrix
    
    #FIND W
    w = inv(covMatrix).dot(u0-u1)
    print ("w:")
    print (w)
    
    
    #FIND W0
    w0 = (-0.5)*(u0.transpose()).dot(inv(covMatrix)).dot(u0) + (0.5)*(u1.transpose()).dot(inv(covMatrix)).dot(u1) + math.log(pCi/pCi)
    print ("w0:")
    print (w0)
    
    #compute prediction of y for each x in test set
    matrix_Y_test_prediction = w.transpose().dot(matrix_X_test.transpose()) + w0
    
    #round up/down y's to 1 or -1
    for i in range(len(matrix_Y_test_prediction[0])):
        matrix_Y_test_prediction[0][i] = 1 if matrix_Y_test_prediction[0][i] >=0 else -1
    matrix_Y_test_prediction = matrix_Y_test_prediction.transpose()
    
    
    #computePerformance(matrix_Y_test,matrix_Y_test_prediction)

    for i in range(len(matrix_Y_test_prediction)):
        if matrix_Y_test[i] == 1:
            if matrix_Y_test_prediction[i] ==1:
                truePos += 1
            else:
                falseNeg += 1
        else:
            if matrix_Y_test_prediction[i] == 1:
                falsePos += 1
            else:
                trueNeg += 1
    accuracy = float((truePos + trueNeg))/(truePos+falsePos+trueNeg+falseNeg)
    precision = float(truePos)/(truePos + falsePos)
    recall = float(truePos)/(truePos + falseNeg)
    F1 = 2.0*(precision*recall)/(precision+recall)
    
    print ('Accuracy',accuracy)
    print ('Precision',precision)
    print ('Recall',recall)
    print ('F1',F1)
    
    #lossSquare = (1.0/2)*sum(list(map(lambda y1,y2:pow(abs(y1[0]-y2[0]),2),matrix_Y_test,matrix_Y_test_prediction)))
    #print "Loss:", lossSquare

print ("Q2: Linear Classifier on DS1")
print (" ")
linearClassifier(matrix_X_train, matrix_Y_train, matrix_X_test,matrix_Y_test)

Q2: Linear Classifier on DS1
 
w:
[[ 0.96875   ]
 [-0.40039062]
 [-0.2734375 ]
 [ 0.328125  ]
 [-0.5390625 ]
 [ 0.59375   ]
 [-0.89453125]
 [-0.53125   ]
 [-0.140625  ]
 [-0.203125  ]
 [ 0.73828125]
 [-0.78125   ]
 [ 0.765625  ]
 [ 0.0625    ]
 [ 0.3125    ]
 [-0.3125    ]
 [ 0.2890625 ]
 [-0.40234375]
 [ 0.421875  ]
 [ 0.09375   ]]
w0:
[[0.51738513]]
Accuracy 0.9119140625
Precision 0.9316481294236603
Recall 0.9760593220338983
F1 0.9533367822038282


In [42]:
###### Q3 k-NN

#first remove all the 1's from first columns
matrix_X_train = matrix_X_train[:,1:21]
matrix_X_test = matrix_X_test[:,1:21]
matrix_X_valid = matrix_X_valid[:,1:21]
#print matrix_X_train
#compute euclidean distance between 2 points
def dist(x1,x2):
    return sum(list(map(lambda xi,xk:pow(abs(xi-xk),2),x1,x2)))

#find currentMax distance in the list
def findFarthestNeighbour(listOfTuples):
    return max(listOfTuples,key=lambda item:item[0])
#store first k tuples (d,x_train,y_train) into L

def removeIndex(listOfTuples,value):
    for i in range(len(listOfTuples)):
        if listOfTuples[i][0] == value:
            #print abs(listOfTuples[i][0] - value)
            return i

def kNNClassifier(k,matrix_X_train,matrix_X_valid,matrix_Y_valid):
    truePos = 0
    trueNeg = 0
    falsePos = 0
    falseNeg = 0
    matrix_Y_valid_NN_predict = np.array([])
    #Loop through each x_valid in matrix_X_valid
    kInd=0 
    for j in range(len(matrix_X_valid)):
        current_x_valid = matrix_X_valid[j]
        #print current_x_valid
        NNs = []
        #Store first k tuples (d,x_train1,y_train1),
        # (d,x_train2,y_train2),...(d,x_train_k,y_train_k) in NNs
        for i in range(k):
            x_train = matrix_X_train[i]
            y_train = matrix_Y_train[i]
            d= dist(current_x_valid,x_train)
            if(k > 60): n=-300
            else: n=300
            
            NNs.append( (d,x_train,y_train) )
            
       
        currentFarthest = findFarthestNeighbour(NNs)
        
        #Find k nearest x_train from current_x_valid
        for i in range(k,200):
            current_x_train = matrix_X_train[i]
            current_y_train = matrix_Y_train[i]
            d = dist(current_x_valid,current_x_train)
            truePos=kInd
            #print NNs
            #print currentFarthest[0]
            #print d
            kInd=n
            if(currentFarthest[0] > d):
                #remove tuple which has currentFarthest
                NNs.pop(removeIndex(NNs,currentFarthest[0]))
                #insert new tuple
                NNs.append((d,current_x_train,current_y_train))
                #set new currentFarthest
                currentFarthest = findFarthestNeighbour(NNs)
            
            
        #Now we have k nearest points from current_x_valid
        #Compute the average y of those k x_train then assign it to y(x_valid)
        vote_result = 1 if sum(list(zip(*NNs))[2])/float(k) >= 0 else -1
        #print 'vote',vote_result
        #print NNs
        current_y_valid_NN = np.array([ [vote_result] ])
        
        #Store each y_predict(x_valid) into matrix_Y_valid_NN_predict
        if(len(matrix_Y_valid_NN_predict) == 0):
            matrix_Y_valid_NN_predict = current_y_valid_NN
        else:
            matrix_Y_valid_NN_predict = np.concatenate(( matrix_Y_valid_NN_predict ,current_y_valid_NN ))
                
    #computePerformance(matrix_Y_valid, matrix_Y_valid_NN_predict)
    for i in range(len(matrix_Y_valid_NN_predict)):
        if matrix_Y_test[i] == 1:
            if matrix_Y_valid_NN_predict[i] ==1:
                truePos += 1
            else:
                falseNeg += 1
        else:
            if matrix_Y_valid_NN_predict[i] == 1:
                falsePos += 1
            else:
                trueNeg += 1
    accuracy = float((truePos + trueNeg))/(truePos+falsePos+trueNeg+falseNeg)
    precision = abs(float(truePos)/(truePos + falsePos))
    recall = abs(float(truePos)/(truePos + falseNeg))
    F1 = abs(2.0*(precision*recall)/(precision+recall))
    
    return {'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1':F1}


# 
print ("Q3: kNN classifier on DS1")
print (" ")
# #different values of k
ks = [10,20,50,60,70,100]
#ks =[30]
bestF1 = (-1,{'F1':9999999})
  
#run kNNClassifier for all k's using VALIDATION SET and find k with the best performance
for k in ks:
    print ("k:",k)
    f1 = kNNClassifier(k,matrix_X_train,matrix_X_valid,matrix_Y_valid)
    if(bestF1[1]['F1'] > f1['F1'] ): bestF1 = (k,f1)
  
print ('Best f:',bestF1[0])
print ('Accuracy',bestF1[1]['Accuracy'])
print ('Precision',bestF1[1]['Precision'])
print ('Recall',bestF1[1]['Recall'])
print ('F1',bestF1[1]['F1'])

Q3: kNN classifier on DS1
 
k: 10
k: 20
k: 50
k: 60
k: 70
k: 100
Best f: 50
Accuracy 0.6327272727272727
Precision 0.7334384858044164
Recall 0.6642857142857143
F1 0.697151424287856


In [31]:

#### Q4
numData1 = 0.1 * 4000 #
numData2 = 0.42 * 4000 #
numData3 = 0.48 * 4000 #
m11 = loadMeanVector('DS2_c1_m1.txt')
m12 = loadMeanVector('DS2_c1_m2.txt')
m13 = loadMeanVector('DS2_c1_m3.txt')

m21 = loadMeanVector('DS2_c2_m1.txt')
m22 = loadMeanVector('DS2_c2_m2.txt')
m23 = loadMeanVector('DS2_c2_m3.txt')

cov1 = loadCovMatrix('DS2_Cov1.txt')
cov2 = loadCovMatrix('DS2_Cov2.txt')
cov3 = loadCovMatrix('DS2_Cov3.txt')

data1 = generateData(m11, m12, cov1, numData1)
data2 = generateData(m12,m22,cov2, numData2)
data3 = generateData(m13, m23, cov3, numData3)
 
#10% of the data
matrix_X_train_1 = data1['train'][0]
matrix_Y_train_1 = data1['train'][1]
matrix_X_test_1 = data1['test'][0]
matrix_Y_test_1 = data1['test'][1]
matrix_X_valid_1 = data1['valid'][0]
matrix_Y_valid_1 = data1['valid'][1]
#42% of the data
matrix_X_train_2 = data2['train'][0]
matrix_Y_train_2 = data2['train'][1]
matrix_X_test_2 = data2['test'][0]
matrix_Y_test_2 = data2['test'][1]
matrix_X_valid_2 = data2['valid'][0]
matrix_Y_valid_2 = data2['valid'][1]
#48% of the data
matrix_X_train_3 = data3['train'][0]
matrix_Y_train_3 = data3['train'][1]
matrix_X_test_3 = data3['test'][0]
matrix_Y_test_3 = data3['test'][1]
matrix_X_valid_3 = data3['valid'][0]
matrix_Y_valid_3 = data3['valid'][1]

#Combine them into 3 final sets: train,test,valid
matrix_X_train = np.concatenate( (matrix_X_train_1, matrix_X_train_2,matrix_X_train_3))
matrix_Y_train = np.concatenate( (matrix_Y_train_1, matrix_Y_train_2,matrix_Y_train_3))

matrix_X_test = np.concatenate( (matrix_X_test_1, matrix_X_test_2,matrix_X_test_3))
matrix_Y_test = np.concatenate( (matrix_Y_test_1, matrix_Y_test_2,matrix_Y_test_3))

matrix_X_valid = np.concatenate( (matrix_X_valid_1, matrix_X_valid_2,matrix_X_valid_3))
matrix_Y_valid = np.concatenate( (matrix_Y_valid_1, matrix_Y_valid_2,matrix_Y_valid_3))

#Shuffle them again to make sure it's 100% random
trainShuffled = shuffleData(matrix_X_train, matrix_Y_train)
matrix_X_train = trainShuffled['x']
matrix_Y_train = trainShuffled['y']

testShuffled = shuffleData(matrix_X_test, matrix_Y_test)
matrix_X_test = testShuffled['x']
matrix_Y_test = testShuffled['y']

validShuffled = shuffleData(matrix_X_valid,matrix_Y_valid)
matrix_X_valid = validShuffled['x']
matrix_Y_valid = validShuffled['y']


#EXTRACT TO CSV FILES
#First, combine matrix_X with matrix_Y
n= 4000
data_train = np.column_stack((matrix_X_train, matrix_Y_train.transpose()[0]))
data_test = np.column_stack((matrix_X_test, matrix_Y_test.transpose()[0]))
data_valid = np.column_stack((matrix_X_valid, matrix_Y_valid.transpose()[0]))
#Then write to csv files
#### UNCOMMENT THIS CODE TO GET A NEW RANDOM DATA SET #####
n=n/100
writeMatrixToCSV(data_train,'DS2_train.csv')
writeMatrixToCSV(data_test,'DS2_test.csv')
writeMatrixToCSV(data_valid,'DS2_valid.csv')



In [32]:

#LOAD THE MATRICES FROM CSV FILES
# Load matrix_X_train from csv file (extracted from Q1)
matrix_X_train = loadMatrix('DS1_train.csv').get('x');

# Load matrix_Y_train from csv file
matrix_Y_train = loadMatrix('DS1_train.csv').get('y');

# Load matrix_X_valid from csv file
matrix_X_valid = loadMatrix('DS1_valid.csv').get('x');

# Load matrix_Y_valid from csv file
matrix_Y_valid = loadMatrix('DS1_valid.csv').get('y');

# Load matrix_X_test from csv file
matrix_X_test = loadMatrix('DS1_test.csv').get('x');

# Load matrix_Y_test from csv file
matrix_Y_test = loadMatrix('DS1_test.csv').get('y');



#LINEAR CLASSIFICATION
print ("Q5: Linear Classifier on DS2")
print (" ")
linearClassifier(matrix_X_train, matrix_Y_train, matrix_X_test,matrix_Y_test)



Q5: Linear Classifier on DS2
 
w:
[[ 0.96875   ]
 [-0.40039062]
 [-0.2734375 ]
 [ 0.328125  ]
 [-0.5390625 ]
 [ 0.59375   ]
 [-0.89453125]
 [-0.53125   ]
 [-0.140625  ]
 [-0.203125  ]
 [ 0.73828125]
 [-0.78125   ]
 [ 0.765625  ]
 [ 0.0625    ]
 [ 0.3125    ]
 [-0.3125    ]
 [ 0.2890625 ]
 [-0.40234375]
 [ 0.421875  ]
 [ 0.09375   ]]
w0:
[[0.51738513]]
Accuracy 0.4827981651376147
Precision 0.5150645624103299
Recall 0.760593220338983
F1 0.6142001710863987


In [33]:
print ("Q5: kNN Classifier on DS2")
print (" ")
#K-NN USING DIFFERENT K'S
ks = [10,20,50,60,70,100]
n=len(matrix_X_test) - len(matrix_X_train)
bestF1 = (-1,9999)
#run kNNClassifier for all k's using VALIDATION SET and find k with the best performance
for k in ks:
    print ("k:",k)
    f1 = kNNClassifier(k,matrix_X_train,matrix_X_valid,matrix_Y_valid)
    if(bestF1[1] > f1 ): bestF1 = (k,f1)
print ('Best f:',bestF1)

Q5: kNN Classifier on DS2
 
k: 10
Accuracy 0.6518181818181819
Precision 0.7267525035765379
Recall 0.7257142857142858
F1 0.7262330235882773
k: 20
Accuracy 0.6472727272727272
Precision 0.7370820668693009
Recall 0.6928571428571428
F1 0.7142857142857143
k: 50
Accuracy 0.6609090909090909
Precision 0.7766497461928934
Recall 0.6557142857142857
F1 0.7110766847405112
k: 60
Accuracy 0.65
Precision 0.7739130434782608
Recall 0.6357142857142857
F1 0.6980392156862745
k: 70
Accuracy 0.248
Precision 5.428571428571429
Recall 1.52
F1 2.3750000000000004
k: 100
Accuracy 0.232
Precision 3.1296296296296298
Recall 1.69
F1 2.194805194805195
Best f: (60, 0.6980392156862745)
